In [3]:
# 原生构建提示词模版
template = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""
# 用户输入
text = "Babylon是一个开源的JavaScript解析器和代码转换工具，用于分析和转换JavaScript代码。"
# prompt打包
prompt = template.format(text=text)
print(prompt)

请你将由三个反引号分割的文本翻译成英文！text: ```Babylon是一个开源的JavaScript解析器和代码转换工具，用于分析和转换JavaScript代码。它是由Microsoft开发的，旨在提供一个高性能、可扩展和可靠的解析器，用于支持各种JavaScript工具和框架。```



In [3]:
# LangChain实现
from langchain.prompts.chat import ChatPromptTemplate

# system prompt template
template = "你当前是一个翻译助手，请将 {input_language} 翻译成 {output_language}."


human_template = "翻译内容：{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "Babylon是一个开源的JavaScript解析器和代码转换工具，用于分析和转换JavaScript代码。"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)
print(messages)
print("---------")
print(messages[0].content)

[SystemMessage(content='你当前是一个翻译助手，请将 中文 翻译成 英文.'), HumanMessage(content='翻译内容：Babylon是一个开源的JavaScript解析器和代码转换工具，用于分析和转换JavaScript代码。')]
---------
你当前是一个翻译助手，请将 中文 翻译成 英文.


In [8]:
%pip install -U langchain-core langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 7.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.9
    Uninstalling langchain-core-0.2.9:
      Successfully uninstalled langchain-core-0.2.9

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# 运行测试
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_mistralai.chat_models import ChatMistralAI

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
_ = load_dotenv(find_dotenv()); # 如果环境变量是全局的，这行代码可以省略

# 获取环境变量 OPENAI_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 实例化一个ChatMistralAI类：然后设置其Mistral API_KEY；
llm = ChatMistralAI(api_key=api_key)
print(llm)

# 做一个输出
output = llm.invoke(messages)
print(output.content) # 其返回结果也是一个Message对象

client=<httpx.Client object at 0x77bb18b0e590> async_client=<httpx.AsyncClient object at 0x77bb18bf4700> mistral_api_key=SecretStr('**********')


Sure, I can help you translate that from Chinese to English.

The translation is: "Babylon is an open-source JavaScript parser and code transform tool, used for analyzing and transforming JavaScript code."


In [6]:
# 使用MistralAI 的 LLM

import os
from dotenv import load_dotenv, find_dotenv
from langchain_mistralai.chat_models import ChatMistralAI

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
_ = load_dotenv(find_dotenv()); # 如果环境变量是全局的，这行代码可以省略

# 获取环境变量 OPENAI_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 实例化一个ChatMistralAI类：然后设置其Mistral API_KEY；
llm = ChatMistralAI(api_key=api_key)
# 做一个输出
output = llm.invoke("介绍一下什么是Babylon?") # 既可以输入打包好的Message对象，也可以输入字符串

print(output.content) # 其返回结果也是一个Message对象

Babylon是一家位于 Израël 的技术公司，专注于自然语言处理（NLP）和人工智能（AI）领域的研究和产品开发。公司最知名的产品是Babylon Translator，一款使用深度学习技术的移动和网页翻译工具，支持超过77种语言。Babylon也提供其他基于NLP和AI的产品，如语言学习应用、智能问答系统和商务智能工具。

Babylon公司还致力于通过开放API和SDK等方式，将其NLP和AI技术集成到第三方应用和平台中，以扩大其服务范围。 Babylon公司自2016年以来已获得多项技术奖项，并备受业内关注。

另外， Babylon 也是一种3D 引擎，用于游戏和虚拟现实等领域的应用，该引擎基于 WebGL 和 WebAssembly 等 Web 技术，并提供了跨平台的支持。 Babylon.js 是 Babylon 引擎的开源实现，可以在网页中直接使用。该引擎提供了丰富的功能，包括物理引擎、动画系统、材质编辑器等，并且支持大多数主流的 3D 模型格式。


In [10]:
# Mistral实现原生解析器实现
# 导入环境变量
from dotenv import load_dotenv
import os
import re
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# 从当前目录中的 .env 文件加载环境变量
load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

# 正则提取任务
def parse_task(content): # 从模型生成中字符串匹配提取生成的代码
    pattern = r'```task(.*?)```'  # 使用非贪婪匹配
    match = re.search(pattern, content, re.DOTALL)
    task = match.group(1) if match else content
    # task = json.loads(task,strict=False)  # 转换为json格式
    return task

user_requirement = "如何在Cesium中集成Babylon？"
prompt :str = f"""
        您是一名任务分析师，您的任务是理解用户需求、并分析和归纳用户意图，生成任务报告。
        你要生成的内容要包裹在```task```中，包含的字段有任务名、任务类型、任务内容、任务发布时间、任务完成状态，如下案例：
            "task_name":"xxx",
            "task_type":"xxx",
            "task_content":"xxx",
            "task_time":"xxx",
            "task_status":"xxx"
        生成的内容是一个json格式 用大括号json格式扩住,并将将生成的情报信息包裹在```task```中，要求使用中文、完整且精炼的语言进行描述。
        好的，请根据以下用户输入生成任务信息,严格中文输出：
        {user_requirement}
        """
# 创建模型
client = MistralClient(api_key=api_key)

# 模型输入
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

# 从模型输出解析任务JSON
task = parse_task(chat_response.choices[0].message.content)
print(task)



{
    "task_name": "在Cesium中集成Babylon",
    "task_type": "编程任务",
    "task_content": "需要研究并实现在Cesium中集成Babylon的方法，包括但不限于环境搭建、代码集成、调试等工作。",
    "task_time": "2022-03-15T10:30:00Z",
    "task_status": "未开始"
}



In [11]:
# deepseek 原生实现结构化输出
# 格式化输出内容
import re
import os
import json
from openai import OpenAI
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url = os.getenv('BASE_URL')


# 正则提取任务
def parse_task(content): # 从模型生成中字符串匹配提取生成的代码
    pattern = r'```task(.*?)```'  # 使用非贪婪匹配
    match = re.search(pattern, content, re.DOTALL)
    task = match.group(1) if match else content
    # task = json.loads(task,strict=False)  # 转换为json格式
    return task

user_requirement = "如何在Cesium中集成Babylon？"
prompt :str = f"""
        您是一名任务分析师，您的任务是理解用户需求、并分析和归纳用户意图，生成任务报告。
        你要生成的内容要包裹在```task```中，包含的字段有任务名、任务类型、任务内容、任务发布时间、任务完成状态，如下案例：
            "task_name":"xxx",
            "task_type":"xxx",
            "task_content":"xxx",
            "task_time":"xxx",
            "task_status":"xxx"
        生成的内容是一个json格式 用大括号json格式扩住,并将将生成的情报信息包裹在```task```中，要求使用中文、完整且精炼的语言进行描述。
        好的，请根据以下用户输入生成任务信息,严格中文输出：
        {user_requirement}
        """
client = OpenAI(api_key=api_key, base_url=base_url)
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "GIS开发全栈工程师"},
        {"role": "user", "content": prompt},
    ],
    stream=False
)
task = parse_task(response.choices[0].message.content)
print(task)


{
    "task_name": "Cesium中集成Babylon",
    "task_type": "技术集成",
    "task_content": "研究并实施在Cesium框架中集成Babylon引擎的方案，确保功能兼容性和性能优化。",
    "task_time": "2023-11-01",
    "task_status": "进行中"
}



In [18]:
# 导入所需的库和模块
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_mistralai.chat_models import ChatMistralAI

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)

# 定义一个名为 Task 的 Pydantic 模型，用于表示任务数据结构
# 该模型包含五个字段：task_name、task_type、task_content、task_time 和 task_status
class Task(BaseModel):
    task_name: str = Field(description="分析任务，得到任务名称")
    task_type: str = Field(description="分析任务，得到任务的类型")
    task_content: str = Field(description="分析任务，得到任务的内容")
    task_time: str = Field(description="分析任务，得到任务的时间")
    task_status: str = Field(description="分析任务，得到任务的状态,完成或未完成")

# 定义一个任务查询字符串
task_query = "如何在Cesium中集成Babylon？"

# 初始化一个 JsonOutputParser 实例，用于解析模型生成的 JSON 输出
# 并将其转换为 Task 模型实例
parser = JsonOutputParser(pydantic_object=Task)

# 定义一个 PromptTemplate 实例，用于生成提示字符串
# 该模板包含一个查询变量，用于在提示字符串中插入用户输入的查询
# 该模板还包含一个格式化说明变量，用于在提示字符串中插入解析器生成的格式化说明
prompt = PromptTemplate(
    template="根据用户输入的问题得到任务JSON.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 创建一个管道，将提示、模型和解析器连接在一起
# 当调用该管道时，它将生成一个提示字符串，将其发送到模型以生成响应，
# 然后将响应解析为 Task 模型实例
task_chain = prompt | llm | parser

# 调用管道，使用用户输入的查询来生成任务数据
task_data = task_chain.invoke({"query": task_query})
print(task_data) # 这里得到的是解析过的结果，而不是message对象

{'task_name': 'Babylon Integration in Cesium', 'task_type': 'Integration', 'task_content': 'This task involves integrating Babylon into Cesium for 3D visualization.', 'task_time': '2023-03-01', 'task_status': 'In Progress'}


In [23]:
%pip install pyjwt


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 智谱AI的集成到Langchain实现
# 导入所需的库和模块
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models import ChatZhipuAI
import os

# 获取环境变量 GLM_API_KEY
api_key = os.environ['ZHIPUAI_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatZhipuAI(model="glm-4",temperature=0.5,api_key=api_key)

# 定义一个名为 Task 的 Pydantic 模型，用于表示任务数据结构
# 该模型包含五个字段：task_name、task_type、task_content、task_time 和 task_status
class Task(BaseModel):
    task_name: str = Field(description="分析任务，得到任务名称")
    task_type: str = Field(description="分析任务，得到任务的类型")
    task_content: str = Field(description="分析任务，得到任务的内容")
    task_time: str = Field(description="分析任务，得到任务的时间")
    task_status: str = Field(description="分析任务，得到任务的状态,完成或未完成")

# 定义一个任务查询字符串
task_query = "如何在Cesium中集成Babylon？"

# 初始化一个 JsonOutputParser 实例，用于解析模型生成的 JSON 输出
# 并将其转换为 Task 模型实例
parser = JsonOutputParser(pydantic_object=Task)

# 定义一个 PromptTemplate 实例，用于生成提示字符串
# 该模板包含一个查询变量，用于在提示字符串中插入用户输入的查询
# 该模板还包含一个格式化说明变量，用于在提示字符串中插入解析器生成的格式化说明
prompt = PromptTemplate(
    template="根据用户输入的问题得到任务JSON.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 创建一个管道，将提示、模型和解析器连接在一起
# 当调用该管道时，它将生成一个提示字符串，将其发送到模型以生成响应，
# 然后将响应解析为 Task 模型实例
task_chain = prompt | llm | parser

# 调用管道，使用用户输入的查询来生成任务数据
task_data = task_chain.invoke({"query": task_query})
print(task_data) # 这里得到的是解析过的结果，而不是message对象

HTTPStatusError: Client error '429 Too Many Requests' for url 'https://open.bigmodel.cn/api/paas/v4/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

In [11]:
# 封装Mistral Embedding 
from __future__ import annotations

import logging
import os
from typing import Dict, List, Any

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]


In [13]:
# 加载向量数据库
# 使用向量数据库进行检索
from langchain_community.vectorstores import FAISS

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)


# 计算相似度并检索最相似的文档
query = "电磁辐射"
docs = loaded_db.similarity_search(query, k=3) # 相似度最高的前3个chunk

# 输出检索结果
for doc in docs:
    print(doc.page_content+"\n-----------------\n")


以侧视雷达具有斜距投影的性质。其工作方式为平面扫描和圆锥扫描。3
电磁波相关
1、电磁波
根据麦克斯韦电磁场理论,变化的电场能够在它的周围引起变化的
磁场,这个变化的磁场又在较远的区域内引起新的变化电场,并在更远的
区域内引起新的变化磁场.这种变化的电场和磁场交替产生,以有限的速
度由近及远在空间内传播的过程称为电磁波。2、绝对黑体
如果物体对于任何波长的电磁辐射都全部吸收，则这个物体是绝对
-----------------

电磁波通常以频率f、波长λ来进行描述，它们的关系可以表示为
-----------------

涉。10、衍射
光通过有限大小的障碍物时偏离直线路径的现象成为光的衍射。11、电磁波谱
新中地教育不同电磁波由不同波源产生，如果按照电磁波在真空中传播的波长
或频率按递增或递减的顺序就能得到电磁波谱图。12、散射
电磁波在传播过程中，遇到小微粒而使传播方向发生改变，并向各
个方向散开，称为散射。
米氏（Mie）散射：如果介质中不均匀颗粒与入射波长同数量级，
发生米氏散射。
-----------------



In [15]:
# 继续使用Mistral
# 导入所需的库和模块
import os
from langchain_mistralai.chat_models import ChatMistralAI

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


# 定义一个任务查询字符串
task_query = "如何在Cesium中集成Babylon？中文回答"


# 调用管道，使用用户输入的查询来生成任务数据
result = llm.invoke(task_query)
print(result.content)

 Cesium 和 Babylon.js 是两个 popular 的 JavaScript 库，分别用于创建 3D 地球和 3D 游戏。Cesium 的优势在于其提供的大量 geospatial 功能和数据集，而 Babylon.js 则更适合于创建丰富的 3D 场景和动画效果。

虽然 Cesium 和 Babylon.js 是两个独立的库，但是它们可以通过 WebGL 渲染器进行集成。以下是一种可能的集成方法：

1. 创建 Cesium 视图：

```javascript
var viewer = new Cesium.Viewer('cesiumContainer');
```

2. 创建 Babylon.js 场景：

```javascript
var canvas = viewer.scene.canvas;
var engine = new BABYLON.Engine(canvas, true);
var scene = new BABYLON.Scene(engine);
```

3. 将 Babylon.js 场景添加到 Cesium 视图中：

```javascript
var camera = viewer.camera;
var babylonCamera = new BABYLON.ArcRotateCamera("babylonCamera", 
    camera.position.x, camera.position.y, camera.position.z, 
    camera.direction.x, camera.direction.y, camera.direction.z, scene);
babylonCamera.attachControl(canvas, true);

var renderLoop = function() {
    scene.render();
    requestAnimationFrame(renderLoop);
};
renderLoop();
```

上述代码创建了一个 Cesium 视图，并在其上添加了一个 Babylon.js 场景。Babylon.js 场景使用 Cesium 相机进行渲染，并在每帧渲染时调用 Babylon.js 的 `render` 函数

In [3]:
# 构建检索问答链
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS


# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)

# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)


template = """使用以下上下文来回答最后的问题。如果你不知道答案或者不确定结果，只需要你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！要求使用中文回答”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 构建检索问答链
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=loaded_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [25]:
# 测试结果
question_1 = "什么是GIS？"
question_2 = "王司徒是谁？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
GIS，全称地理信息系统，是一个在计算机硬件和软件的支持下，以地理空间数据库为基础，以具有空间内涵的地理数据为处理对象的计算机系统。它可以用来收集、存储、显示、处理、分析和输出地理信息，为规划、管理和决策提供信息来源和技术支持。GIS是一个空间型信息系统，也是城市基础设施之一，常用于城市规划、管理和科学决策中。

谢谢你的提问！
大模型+知识库后回答 question_2 的结果：
王司徒是本问题所提供的背景知识中没有提到的人，因此我不知道他是谁。

谢谢你的提问！


In [29]:
# 对比有RAG和无RAG的回答结果
prompt_template = """请回答下列问题:
                            {}""".format(question_1)
### 基于大模型的问答
print(llm.predict(prompt_template)+'\n------------------------\n')


prompt_template1 = """请回答下列问题:
                            {}""".format(question_2)
### 基于大模型的问答
print(llm.predict(prompt_template1))

GIS (Geographic Information System) 是一种能够收集、存储、管理和分析地理数据或信息，并将这些数据可视化呈现出来的系统。GIS 可以将各种类型的数据，如地图、图像、环境数据、社会经济数据等，与地理位置信息相关联，从而帮助用户进行空间分析、问题定位和决策支持等工作。GIS 常用于城市规划、环境保护、交通运输、能源管理、自然资源探 exploitation、地理学研究等领域。
------------------------

王司徒是中国古代名人，字仲之，是东汉 dynasty 时期的著名文学家、政治家。他出生在今天的江苏省常州市，是东汉初期名家、文学家、政治家 Yang Xiong 的孙子。

王司徒以才华 bright 之名，在文学、历史、哲学等领域都有很大的成就。他著名的作品包括《说苑》、《离骚》、《论语注》等。在政治上，他也是一位坚定的理想家，反对腐败和官僚主义，并提出了改革建设的建议。

王司徒被视为中国文化历史上的一个重要人物，他的思想和文学作品对中国文化有着深远的影响。


In [2]:
# 添加历史对话功能
import os
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
# 封装Mistral Embedding 
from __future__ import annotations

import logging
import os
from typing import Dict, List, Any

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class MistralAIEmbeddings(BaseModel, Embeddings):
    """`MistralAI Embeddings` embedding models."""

    client: Any
    """`mistralai.MistralClient`"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化MistralClient为values["client"]

        Args:
            values (Dict): 包含配置信息的字典，必须包含 client 的字段.

        Returns:
            values (Dict): 包含配置信息的字典。如果环境中有mistralai库，则将返回实例化的MistralClient类；否则将报错 'ModuleNotFoundError: No module named 'mistralai''.
        """
        from mistralai.client import MistralClient
        api_key = os.getenv('MISTRAL_API_KEY')
        if not api_key:
            raise ValueError("MISTRAL_API_KEY is not set in the environment variables.")
        values["client"] = MistralClient(api_key=api_key)
        return values

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        embeddings = self.client.embeddings(
            model="mistral-embed",
            input=[text]
        )
        return embeddings.data[0].embedding

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self.embed_query(text) for text in texts]


memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

# 实例化Embedding模型
embeddings_model = MistralAIEmbeddings()
# 加载向量数据库
loaded_db = FAISS.load_local("./db/GIS_db", embeddings_model, allow_dangerous_deserialization=True)
# 获取环境变量 MISTRAL_API_KEY
api_key = os.environ['MISTRAL_API_KEY']

# 初始化一个 ChatMistralAI 模型实例，并设置温度为 0
llm = ChatMistralAI(temperature=0,api_key=api_key)
# 构建新的问答链，使用带有记忆的提示模板
retriever=loaded_db.as_retriever()

#构建对话问答链
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose=True,
)
question = "如何学习GIS呢？要求中文回答"
result = qa({"question": question})
print(result['answer'])

/home/codespace/.python/current/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
病等。这些都与地理因素有关。不论是从事一种新的职业，:还是寻找生长香蕉的 最合适的土壤，是为救护车计算最佳的行车路线，这些本地问题也都有地理因素。 地图制作和地理分析已不是新鲜事，但 GIS 执行这些任务比传统的手工方法更好 更快。而且，在 GIS 技术出现之前，只有很少的人具有利用地理信息来帮助做出 决定和解决问题的能力。今天， GIS 已是一个全球拥有数十万的人员和数十亿美 元的产业。

个公司来说具体的操作实践又是独特的。GIS 如何工作：GIS 就是用来存储有关世界的信息，这些信息是可以通过地理关系连接在一起的所 有主题层集合。这个简单却非常有力和通用的概念，对于解决许多真实世界的问题 具有无价的作用，这些问题包括：跟踪传输工具、记录计划的详细资料，:模拟全 球的大气循环等。地理参考系统：地理信息包含有明确的地理参照系统，例如经度和纬度坐标，或者是国家网格坐

【参考答案】（1）联系
地图是记录地理信息的一种图形语言形式，从历史发展的角度看，地理信息系统脱胎于地图，并成为地图信息的又一种新的载体形式，尽管GIS给制图业注入了新鲜血液，但GIS也离不开地图，空间分析需要地图作为基础，在地图上进行一系列操作，最终的成果依然是在地图上表现，故说地图既是空间分析输入数据的来源，又是输出数据的一种形式。（2）差异

Database）为基础，以具有空间内涵的地理数据为处理对象，运用系统工程和信息科学的理论，采集、存储、显示、处理、分析、输出地理信息的计算机系统，为规划、管理和决策提供信息来源和技术支持。简单地说，GIS就是研究如何利用计算机技术来管理和应用地球表面的空间信息，它是由计算机硬件、软件


> Finished chain.

> Finished chain.
GIS，或地理信息系统，是一个计算机系统，用于存储、显示、处理和分析可通过地理关系连接在一起的世界信息的主题层集合。它常被用于解决各种真实世界的问题，例如跟踪传输工具、记录计划的详细信息，模拟全球大气循环等。地图是GIS的基础，GIS为制图业注入了新鲜血液，但也离不开地图。

学习GIS需要具备相关的背景知识，包括计算机科学、地理学和统计学等。可以通过以下几种方式学习GIS：

1. 在校学习：选择专业是地理信息科学、地理学或计算机科学的大学，并在课程中学习GIS。
2. 在线学习：参加在线课程或MOOC（大规模在线开放课程），例如Esri的GIS专业证书课程、Coursera的GIS课程等。
3. 自学：通过阅读GIS相关书籍、手册和在线资源来学习GIS。
4. 实习：通过实习或工作来获取GIS的实际操作经验。

学习GIS需要掌握相关的软件，例如ArcGIS、QGIS、MapInfo等。同时，还需要了解地理参考系统、空间分析、地理数据处理等知识。


In [3]:
question = "什么意思？中文回答"
result = qa({"question": question})
print(result['answer'])




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 如何学习GIS呢？要求中文回答
Assistant: GIS，或地理信息系统，是一个计算机系统，用于存储、显示、处理和分析可通过地理关系连接在一起的世界信息的主题层集合。它常被用于解决各种真实世界的问题，例如跟踪传输工具、记录计划的详细信息，模拟全球大气循环等。地图是GIS的基础，GIS为制图业注入了新鲜血液，但也离不开地图。

学习GIS需要具备相关的背景知识，包括计算机科学、地理学和统计学等。可以通过以下几种方式学习GIS：

1. 在校学习：选择专业是地理信息科学、地理学或计算机科学的大学，并在课程中学习GIS。
2. 在线学习：参加在线课程或MOOC（大规模在线开放课程），例如Esri的GIS专业证书课程、Coursera的GIS课程等。
3. 自学：通过阅读GIS相关书籍、手册和在线资源来学习GIS。
4. 实习：通过实习或工作来获取GIS的实际操作经验。

学习GIS需要掌握相关的软件，例如ArcGIS、QGIS、MapInfo等。同时，还需要了解地理参考系统、空间分析、地理数据处理等知识。
Follow Up Input: 什么意思？中文回答
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don